# Installing Packages

In [ ]:
!pip install -U transformers
!pip install sentencepiece
!pip install accelerate
!pip install bitsandbytes

# Importing dependencies

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
import pandas as pd
import os
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch
from transformers import BitsAndBytesConfig


csv_data = CSVLoader(file_path='-Your-filepath-')

# Initalizing model functionality

In [ ]:
#defining the model
quantization_config = BitsAndBytesConfig(llm_int4_enable_fp32_cpu_offload=True)

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_4bit=True,
    quantization_config=quantization_config,
    offload_folder="offload",
    torch_dtype=torch.float16,
    device_map='auto',
)

base_model.tie_weights()

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([csv_data])

In [ ]:
chain = RetrievalQA.from_chain_type(llm=local_llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

# Implemented usage of model
can be implemented in a similar way for projects with UI


In [ ]:
query = "-own query-"
response = chain({"question": query})
print(response['result'])